In [1]:
""" ACTIVE JUPYTER NOTEBOOK TO BATCH RUN UNIT MATCHING ALGORITHM """

# To be able to make edits to repo without having to restart notebook
%load_ext autoreload
%autoreload 2


In [2]:
# Outside imports
import os, sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Set necessary paths / make project path = ...../neuroscikit/
unit_matcher_path = os.getcwd()
prototype_path = os.path.abspath(os.path.join(unit_matcher_path, os.pardir))
project_path = os.path.abspath(os.path.join(prototype_path, os.pardir))
lab_path = os.path.abspath(os.path.join(project_path, os.pardir))
sys.path.append(project_path)
os.chdir(project_path)
print(project_path)

# Internal imports

# Read write modules
from x_io.rw.axona.batch_read import make_study
from _prototypes.unit_matcher.read_axona import read_sequential_sessions, temp_read_cut
from _prototypes.unit_matcher.write_axona import format_new_cut_file_name

# Unit matching modules
from _prototypes.unit_matcher.main import format_cut, run_unit_matcher, map_unit_matches_first_session, map_unit_matches_sequential_session
from _prototypes.unit_matcher.session import compare_sessions
from _prototypes.unit_matcher.waveform import time_index, derivative, derivative2, morphological_points

c:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit


In [ ]:
""" ONLY EDIT THE SETTINGS IN THE NEXT TWO CELLS """

In [3]:
""" If a setting is not used for your analysis (e.g. smoothing_factor), just pass in an arbitrary value or pass in 'None' """
STUDY_SETTINGS = {

    'ppm': 511,  # EDIT HERE

    'smoothing_factor': 3, # EDIT HERE

    'useMatchedCut': False,  # EDIT HERE, set to False if you want to use runUnitMatcher, set to True after to load in matched.cut file
}


# Switch devices to True/False based on what is used in the acquisition (to be extended for more devices in future)
device_settings = {'axona_led_tracker': True, 'implant': True} 

# Make sure implant metadata is correct, change if not, AT THE MINIMUM leave implant_type: tetrode
implant_settings = {'implant_type': 'tetrode', 'implant_geometry': 'square', 'wire_length': 25, 'wire_length_units': 'um', 'implant_units': 'uV'}

# WE ASSUME DEVICE AND IMPLANT SETTINGS ARE CONSISTENCE ACROSS SESSIONS

# Set channel count + add device/implant settings
SESSION_SETTINGS = {
    'channel_count': 4, # EDIT HERE, default is 4, you can change to other number but code will check how many tetrode files are present and set that to channel copunt regardless
    'devices': device_settings, # EDIT HERE
    'implant': implant_settings, # EDIT HERE
}

STUDY_SETTINGS['session'] = SESSION_SETTINGS

settings_dict = STUDY_SETTINGS

In [4]:
# EDIT HERE --> change to path to your data, can ignore lab_path and put full file path to a folder as: r'path_to_data'
# data_dir = lab_path + r'\neuroscikit_test_data\20180502-ROUND-3000'
# data_dir = lab_path + r'\neuroscikit_test_data\single_sequential'
# data_dir = lab_path + r'\neuroscikit_test_data\Outputs'
# data_dir = lab_path + r'\neuroscikit_test_data\20170315-270-3525_Test' 
# data_dir = lab_path + r'\neuroscikit_test_data\RadhaData\Data\highPHF'
# data_dir = lab_path +  r'\neuroscikit_test_data\AD_group\Animal1'
# data_dir = lab_path +  r'\neuroscikit_test_data\LEC_sample_single_tet'
# data_dir = lab_path + r'\neuroscikit_test_data\20180530-ROUND-3300-1X2B3A'
# data_dir = lab_path + r'\neuroscikit_test_data\LEC_odor\Non-AD'
# data_dir = lab_path + r'\neuroscikit_test_data\LEC_odor\AD'
data_dir = lab_path + r'\neuroscikit_test_data\ANT-119a-6_20170613\ANT-119a-6_20170613'
# data_dir = lab_path + r'\neuroscikit_test_data\3_ses_folder'

# data_dir = lab_path + r'\neuroscikit_test_data\Test_File_Andrew-20221216T163437Z-001\Test_File_Andrew'


# To use in unit matching
settings_dict_unmatched = settings_dict
settings_dict_unmatched['useMatchedCut'] = False

In [ ]:
animal = study.get_animal_by_id('Odor_119a-6_tet2')
print(np.unique(animal.sessions['session_26'].get_spike_data()['spike_cluster'].cluster_labels))
print(animal.sessions['session_26'].get_cell_data()['cell_ensemble'].get_label_ids())

In [5]:
study = make_study([data_dir], settings_dict_unmatched)
study.make_animals()

SKIPPING, NO FILES FOUND FOR SESSION WITH SIGNATURE: c:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit_test_data\LEC_odor\AD/Odor_119a-6/All_PRISM_Sessions_iter
SKIPPING, NO FILES FOUND FOR SESSION WITH SIGNATURE: c:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit_test_data\LEC_odor\AD/Odor_119a-6/All_PRISM_Sessions_iter_2/PRISM_Session_ANT-119a-6_20170606-3100-odorxba_0/Tetrode
SKIPPING, NO FILES FOUND FOR SESSION WITH SIGNATURE: c:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit_test_data\LEC_odor\AD/sample/All_PRISM_Sessions_iter_7/PRISM_Session_ANT-119a-6_20170718-2125-odorxab_2/Tetrode
SKIPPING, NO FILES FOUND FOR SESSION WITH SIGNATURE: c:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit_test_data\LEC_odor\AD/Odor_119a-6/All_PRISM_Sessions_iter_4/PRISM_Session_ANT-119a-6_20170619-2375-odorxba
SKIPPING, NO FILES FOUND FOR SESSION WITH SIGNATURE: c:\Users\aaoun\OneDrive - cumc.columb

In [23]:
for animal in study.animals:
    print(animal.animal_id)
    print(animal.sessions)

B6-LEC1_20160304_tet1
{'session_1': <library.study_space.Session object at 0x000001D84C0977F0>, 'session_2': <library.study_space.Session object at 0x000001D81BE2E0B0>}
B6-LEC1_20160304_tet2
{'session_1': <library.study_space.Session object at 0x000001D84C097880>, 'session_2': <library.study_space.Session object at 0x000001D859D1AA70>}
B6-LEC1_20160304_tet3
{'session_1': <library.study_space.Session object at 0x000001D84C097DF0>, 'session_2': <library.study_space.Session object at 0x000001D859D197E0>}
B6-LEC1_20160304_tet4
{'session_1': <library.study_space.Session object at 0x000001D84C097FD0>, 'session_2': <library.study_space.Session object at 0x000001D859D1AE00>}
B6-LEC1_20160125_tet1
{'session_1': <library.study_space.Session object at 0x000001D859D1B340>, 'session_2': <library.study_space.Session object at 0x000001D84C095150>}
B6-LEC1_20160125_tet2
{'session_1': <library.study_space.Session object at 0x000001D859D189D0>, 'session_2': <library.study_space.Session object at 0x00000

In [12]:
ct = []
for animal in study.animals:
    ct.append(animal.animal_id)
print(len(ct), len(np.unique(ct)))

40 40


In [20]:
# Run unit matching on non-matched study, will save new matched cut file. 
# First all data is loaded ('Animal ID set': Animal1_tet1, Animal1_tet2, etc..)
# Then sessions PER animal and spikes per cell are sorted and  ('Session data added (to animal), spikes sorted by cell': Animal1_tet1_ses1, Animal1_tet1_ses2, etc..)
# Then unit matching begins Animal1_tet1_ses1_cell_1 vs Animal1_tet1_ses2_cell_1, Animal1_tet1_ses1_cell_1 vs Animal1_tet1_ses2_cell_2, Animal1_tet1_ses1_cell_1 vs Animal1_tet1_ses2_cell_3, etc...
unmatched_study = run_unit_matcher([data_dir], settings_dict_unmatched)
print('COMPLETED UNIT MATCHING')

Running Unit Matcher. Starting Time Tracker.
SKIPPING, NO FILES FOUND FOR SESSION WITH SIGNATURE: c:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit_test_data\3_ses_folder/B6-LEC1_201603
SKIPPING, NO FILES FOUND FOR SESSION WITH SIGNATURE: c:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit_test_data\3_ses_folder/B6-LEC1_201602
SKIPPING, NO FILES FOUND FOR SESSION WITH SIGNATURE: c:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit_test_data\3_ses_folder/B6-LEC1_201601
Animal ID set
Animal ID set
Animal ID set
Animal ID set
Animal ID set
Animal ID set
Animal ID set
Animal ID set
Animal ID set
Animal ID set
Animal ID set
Animal ID set
Animal ID set
Animal ID set
Animal ID set
Animal ID set
Animal ID set
Animal ID set
Animal ID set
Animal ID set
Animal ID set
Animal ID set
Animal ID set
Animal ID set
Animal ID set
Animal ID set
Animal ID set
Animal ID set
Animal ID set
Animal ID set
Animal ID set
Animal ID set
Animal

c:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit\_prototypes\unit_matcher\waveform.py:321: RuntimeWarning: invalid value encountered in double_scalars
  return np.sum((data - mean)**4) / (n * std**4)
c:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit\_prototypes\unit_matcher\waveform.py:302: RuntimeWarning: invalid value encountered in double_scalars
  return np.sum((data - mean)**3) / (n * std**3)
c:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit\_prototypes\unit_matcher\main.py:96: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  
c:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit\_prototypes\unit_matcher\main.py:246: VisibleDeprecationWarning: Creating an ndarray fro

New here
[1 2 3 4] [1 2 3 4]
{'cut': 'c:\\Users\\aaoun\\OneDrive - cumc.columbia.edu\\Desktop\\HussainiLab\\neuroscikit_test_data\\3_ses_folder/B6-LEC1_20160304/B6-LEC1_20160304-0_1.cut', 'tet': 'c:\\Users\\aaoun\\OneDrive - cumc.columbia.edu\\Desktop\\HussainiLab\\neuroscikit_test_data\\3_ses_folder/B6-LEC1_20160304/B6-LEC1_20160304-0.1', 'pos': 'c:\\Users\\aaoun\\OneDrive - cumc.columbia.edu\\Desktop\\HussainiLab\\neuroscikit_test_data\\3_ses_folder/B6-LEC1_20160304/B6-LEC1_20160304-0.pos', 'ppm': 300.0}
339
Using post PCA feature array
(4,) (4,) 97
4 4
JSD: 0.43186996204507744
JSD: 0.45758464069720756
JSD: 0.40073265691102544
JSD: 0.43726080239353066
JSD: 0.4329077450029643
JSD: 0.47461198627649676
JSD: 0.4016388940835997
JSD: 0.4162633535268366
JSD: 0.41579677496219747
JSD: 0.4557139614347093
JSD: 0.41478325341155586
JSD: 0.4347926300037498
JSD: 0.4274713044175871
JSD: 0.4822353803067975
JSD: 0.40895877066888986
JSD: 0.4260887618743274
[[0.43186996 0.45758464 0.40073266 0.4372608 ]

c:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit\_prototypes\unit_matcher\waveform.py:321: RuntimeWarning: invalid value encountered in double_scalars
  return np.sum((data - mean)**4) / (n * std**4)
c:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit\_prototypes\unit_matcher\waveform.py:302: RuntimeWarning: invalid value encountered in double_scalars
  return np.sum((data - mean)**3) / (n * std**3)
c:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit\_prototypes\unit_matcher\main.py:96: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  
c:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit\_prototypes\unit_matcher\main.py:246: VisibleDeprecationWarning: Creating an ndarray fro

New here
[1 2 3 4] [1 2 3 4]
{'cut': 'c:\\Users\\aaoun\\OneDrive - cumc.columbia.edu\\Desktop\\HussainiLab\\neuroscikit_test_data\\3_ses_folder/B6-LEC1_20160304/B6-LEC1_20160304-0_2.cut', 'tet': 'c:\\Users\\aaoun\\OneDrive - cumc.columbia.edu\\Desktop\\HussainiLab\\neuroscikit_test_data\\3_ses_folder/B6-LEC1_20160304/B6-LEC1_20160304-0.2', 'pos': 'c:\\Users\\aaoun\\OneDrive - cumc.columbia.edu\\Desktop\\HussainiLab\\neuroscikit_test_data\\3_ses_folder/B6-LEC1_20160304/B6-LEC1_20160304-0.pos', 'ppm': 300.0}
366
Using post PCA feature array
(5,) (4,) 97
5 4
JSD: 0.5489337120932261
JSD: 0.6598061576530191
JSD: 0.5549497239450314
JSD: 0.5678847384967858
JSD: 0.6780848578722971
JSD: 0.780097999550744
JSD: 0.6777763584483437
JSD: 0.6909730402679748
JSD: 0.4139375220667607
JSD: 0.46057222054016733
JSD: 0.44255330533278453
JSD: 0.4451656120853735
JSD: 0.5930786737476846
JSD: 0.6880601333146803
JSD: 0.5979217152200468
JSD: 0.6077803980508885
JSD: 0.44500589589382733
JSD: 0.5449808870428693
JSD:

...

New here
[1 2 3 4] [1 2 3 4]
{'cut': 'c:\\Users\\aaoun\\OneDrive - cumc.columbia.edu\\Desktop\\HussainiLab\\neuroscikit_test_data\\3_ses_folder/B6-LEC1_20160212/B6-LEC1_20160212-0_3.cut', 'tet': 'c:\\Users\\aaoun\\OneDrive - cumc.columbia.edu\\Desktop\\HussainiLab\\neuroscikit_test_data\\3_ses_folder/B6-LEC1_20160212/B6-LEC1_20160212-0.3', 'pos': 'c:\\Users\\aaoun\\OneDrive - cumc.columbia.edu\\Desktop\\HussainiLab\\neuroscikit_test_data\\3_ses_folder/B6-LEC1_20160212/B6-LEC1_20160212-0.pos', 'ppm': 300.0}
511
Using post PCA feature array
(5,) (4,) 97
5 4
JSD: 0.4034877374677746
JSD: 0.4605392492061244
JSD: 0.4634689572967376
JSD: 0.44834354618837285
JSD: 0.4403603759034635
JSD: 0.40664047807185266
JSD: 0.4692483847006229
JSD: 0.46305667457388167
JSD: 0.4129172329886717
JSD: 0.46302041286428464
JSD: 0.44035802928079376
JSD: 0.45088105595108147
JSD: 0.6594715676256667
JSD: 0.6527246999638702
JSD: 0.625200985917529
JSD: 0.6310571809368276
JSD: 0.4393308319266785
JSD: 0.4633500349267548
J

c:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit\_prototypes\unit_matcher\waveform.py:321: RuntimeWarning: invalid value encountered in double_scalars
  return np.sum((data - mean)**4) / (n * std**4)
c:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit\_prototypes\unit_matcher\waveform.py:302: RuntimeWarning: invalid value encountered in double_scalars
  return np.sum((data - mean)**3) / (n * std**3)
c:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit\_prototypes\unit_matcher\main.py:96: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  
c:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit\_prototypes\unit_matcher\main.py:246: VisibleDeprecationWarning: Creating an ndarray fro

New here
[1 2 3 4 5] [1 2 3 4 5]
{'cut': 'c:\\Users\\aaoun\\OneDrive - cumc.columbia.edu\\Desktop\\HussainiLab\\neuroscikit_test_data\\3_ses_folder/B6-LEC1_20160212/B6-LEC1_20160212-0_4.cut', 'tet': 'c:\\Users\\aaoun\\OneDrive - cumc.columbia.edu\\Desktop\\HussainiLab\\neuroscikit_test_data\\3_ses_folder/B6-LEC1_20160212/B6-LEC1_20160212-0.4', 'pos': 'c:\\Users\\aaoun\\OneDrive - cumc.columbia.edu\\Desktop\\HussainiLab\\neuroscikit_test_data\\3_ses_folder/B6-LEC1_20160212/B6-LEC1_20160212-0.pos', 'ppm': 300.0}
487
Using post PCA feature array
(4,) (5,) 97
4 5
JSD: 0.4285825145890856
JSD: 0.42324423913914
JSD: 0.5608664354440026
JSD: 0.8108037559360582
JSD: 0.5191379979410604
JSD: 0.4408233196195572
JSD: 0.4439641365029499
JSD: 0.496622027771838
JSD: 0.7154669646217814
JSD: 0.4542715857952991
JSD: 0.49006954447846707
JSD: 0.47808442643495447
JSD: 0.45262333550868306
JSD: 0.7291812287689995
JSD: 0.4659486752366459
JSD: 0.5281108711605639
JSD: 0.5424819346572145
JSD: 0.49282427485648084
J

c:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit\_prototypes\unit_matcher\waveform.py:321: RuntimeWarning: invalid value encountered in double_scalars
  return np.sum((data - mean)**4) / (n * std**4)
c:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit\_prototypes\unit_matcher\waveform.py:302: RuntimeWarning: invalid value encountered in double_scalars
  return np.sum((data - mean)**3) / (n * std**3)
c:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit\_prototypes\unit_matcher\main.py:246: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  indiv_ses_feats.append(np.array(indiv_cell_feats))


UnboundLocalError: local variable 'comparison' referenced before assignment

In [ ]:
# Quick check that session file are grouped correctly + ordered in time
tets = []
for ses in unmatched_study.animals[0].sessions:
    print(unmatched_study.animals[0].sessions[ses].session_metadata.file_paths)
    tets.append(unmatched_study.animals[0].sessions[ses].session_metadata.file_paths['tet'])
    print(unmatched_study.animals[0].sessions[ses].datetime)

In [ ]:
sorted(tets)

In [ ]:
# New settings dictionary indicating the matched cut file should now be used for file loading
settings_dict_matched = settings_dict
settings_dict_matched['useMatchedCut'] = True

# Load new study but using labels from matched cut file
matched_study = make_study([data_dir], settings_dict_matched)

# Sort spikes by cell and order sessions sequentially
matched_study.make_animals()

In [ ]:
# Look at matched cell ids from new cut files
for animal in matched_study.animals:
    print('New Animal')
    for session in animal.sessions.values():
        print('Session date & time: ' + str(session.datetime))
        print('Matched cell ids: ' + str(np.unique(session.get_spike_data()['spike_cluster'].cluster_labels)))

In [ ]:
# Pull out session classes and cell ensembles from matched and unmatched studies for plotting

session1 = matched_study.animals[0].sessions['session_1']
ensemble1 = matched_study.animals[0].ensembles['session_1']
session2 = matched_study.animals[0].sessions['session_2']
ensemble2 = matched_study.animals[0].ensembles['session_2']

unmatched_ensembles1 = unmatched_study.animals[0].ensembles['session_1']
unmatched_ensembles2 = unmatched_study.animals[0].ensembles['session_2']

In [ ]:
""" Plot session 1 (left) & session 2 (right) MATCHED units """

pair_count = len(ensemble1.get_label_ids())

for i in range(pair_count):
# for i in range(2):

    fig = plt.figure(figsize=(6,12))

    axes = []

    waveforms1 = ensemble1.cells[i].signal
    waveforms2 = ensemble2.cells[i].signal

    avg_waveforms1 = np.mean(waveforms1, axis=0)
    avg_waveforms2 = np.mean(waveforms2, axis=0)

    assert waveforms1.shape[1] == avg_waveforms1.shape[0]

    for j in range(0,avg_waveforms1.shape[0]*2,2):
        ax1 = plt.subplot(avg_waveforms1.shape[0],2,j+1)
        ax2 = plt.subplot(avg_waveforms1.shape[0],2,j+2)

        ax1.plot(waveforms1[:,int(j/2)].T, color='gray', lw=0.5, alpha=0.5)
        ax2.plot(waveforms2[:,int(j/2)].T, color='gray', lw=0.5, alpha=0.5)

        ax1.plot(avg_waveforms1[int(j/2)], color='k', lw=2)
        ax2.plot(avg_waveforms2[int(j/2)], color='k', lw=2)

        ax1.set_title('Channel ' + str(int(j/2+1)))
        ax2.set_title('Channel ' + str(int(j/2+1)))

        axes.append(ax1)
        axes.append(ax2)

    for ax in axes:
        ax.set_xlabel('Bin Number')
        ax.set_ylabel('Waveform')

    fig.suptitle('Session 1 (left) & 2 (right) - Unit ' + str(i+1))

    fig.tight_layout()
    plt.show()
